## [Lab 4 Part II] Blockchain Basics

In Part I, you have implemented basic cryptographic functions. In Part II, you will manage transactions and blocks.

Part II consists of four parts.

* Model: A blockchain has blocks of transactions.  You will implement functions to create and manage them.
* DBManager: BLENDS uses a RDBMS called SQLite.  You will implement functions to store, index, and search blockchain data.
* Verifier/Validator: You will implement functions to verify if blocks and transactions are formatted correctly.
* Miner: You will implement functions to mine a new block.

We have mined a few blocks and put them in "scenario.db".  The first block in the block chain is identified with a parent block hash value of 0xdecaf.


In [ ]:
from blends.node import crypto, util

sender = crypto.load_secret_key('key0.json')
receiver = crypto.load_secret_key('key1.json')

## Section 1: Model

All blocks and transactions are objects. You will learn how to manage transactions and blocks one by one in this section.

### Tutorial 1 : Create a new transaction

BLENDS offers the following API to create a new transaction.

```python
def new_transaction(version: str, sender: str, receiver: str, timestamp: str, amount: int):
```


In [ ]:
from blends.node.model import Transaction
transaction = Transaction.new_transaction("test", hex(sender.n), hex(receiver.n), util.now(), 10)

This transaction has not been digital signed and must be signed with the sender's private key before use.

### Tutorial 2 : Transaction Hash

The transaction hash is a unique identifier for each transaction.  You compute the transaction hash by applying a hash function to the transaction payload.  Below is the code for computing the transaction hash.

**You must use `get_payload` to fetch the transaction payload.**

In [ ]:
payload = transaction.get_payload()
hash_ = crypto.get_hash(payload)
transaction.set_hash(hash_)

print(transaction)

### Exercise 1 : Sign a transaction

You have learned how to compute the hash of a transaction.  Now you will sign the `transaction`.

```
def set_sign(self, sign: str):
```


In [ ]:
sign = None #<- should be implemented

transaction.set_sign(sign)

### Tutorial 3 : Create a new block

You have created a transaction, computed its hash, and signed it.  Now let's create a block.  In BLENDS has defined a  `Block` class. Use the following API to create a new block.

```python
def new_block(version: str, parent: str, timestamp: str, miner: str, difficulty: int):
```


In [ ]:
from blends.node.model import Block
block = Block.new_block("test", None, util.now(), hex(sender.n), 0)

### Tutorial 4: Nonce

Every block will have a nonce. You change the value of this nonce later in mining.  Use the following API to change the value of the nonce.

```python
def set_nonce(self, nonce: int)
```


In [ ]:
block.set_nonce(1)

### Tutorial 5: Add a transaction to a block

A single block may contain multiple transactions.  In BLENDS, you can add transactions to `blocks.txt`.


In [ ]:
block.txs.append(transaction)

### Exercise 2 : Block Hash

Every block has a unique hash value. In BLENDS, use the following API to set the hash value.

```python
def set_hash(self, hash)
```

As with transactions, you must use the `get_payload` function to get the payload of a block. Now compute the hash of the block.

Below compute the hash of the block you just created above.

In [ ]:
payload = block.get_payload()
print(payload)

## should be implemented



So far you have learned to create transactions and blocks and set their hash values.  Next, you will implement blockchain operations on `transactions` and `blocks`.

## Section 2: DBManager

BLENDS uses SQLite to store and index transactions and blocks.

In this section, you will implement `DBManager` in BLENDS.

You have not implemented mining functions, and thus BLENDS offers a preloaded database for you to work with.


In [ ]:
# Initiazation code for DBManager
from blends.node.blockchain.dbmanager import DBManager
dbmanager = DBManager("sqlite:///scenario.db")

### Problem 1 : `DBManager.search_block`

Implement `def search_block(self, block_hash: str)` function in [dbmanager.py](/edit/blends/node/blockchain/dbmanager.py)

* Given a hash value, search the block with the hash value.
* If the search returns successful, return the `Block` object; otherwise, return `None`.
* Using `session.query(Block).all()`, iterate through the entire block chain.
* If you are familiar with DB and ORM, feel free to use ORM.

In [ ]:
# Test  
session = dbmanager.get_session()
block_set = session.query(Block).all()
for block in block_set:
    header = block.get_header()
    if(block == dbmanager.search_block(header["hash"])):
        print("PASS")
    else:
        print("FAIL")

### Problem 2: `DBManager.get_height`

Implement `def get_height(self, block_hash: str)` function in [dbmanager.py](/edit/blends/node/blockchain/dbmanager.py)

* Now let's compute the height of the block. BLENDS does not use the Merkle tree, thus the height equals the length in the blockchain.
* If successful, the function returns `height : int`; otherwise, `None`.
* The genesis block with no parent has `height` of 0.
* Iterate through the blocks using `session.query(Block).all()`.
* Feel free to use ORM.


In [ ]:
# Test  
session = dbmanager.get_session()
block_set = session.query(Block).all()
height_set = [0, 1, 2, 3, 4]
for i in range(5):
    header = block_set[i].get_header()
    if(height_set[i] == dbmanager.get_height(header["hash"])):
        print("PASS")
    else:
        print("FAIL")

### Problem 3 : `DBManager.get_current`

Implement `def get_current(self)` function in [dbmanager.py](/edit/blends/node/blockchain/dbmanager.py)

* Now let's fetch the highest block in the database.
* If there are more than one block with the same height, either will do.
* You may use functions implemented above.
* You can iterate through the chain using `session.query(Block).all()`.
* Again, feel free to use ORM.


In [ ]:
# Test  
session = dbmanager.get_session()
block_set = session.query(Block).all()
if(block_set[4] == dbmanager.get_current()):
    print("PASS")
else:
    print("FAIL")

### Problem 4 : `DBManager.get_longest`

Implement `def get_longest(self)` function in [dbmanager.py](/edit/blends/node/blockchain/dbmanager.py)

* Now let's fetch the longest chain in the database.  Just get the longest chain.
* If there are more than one longest chain with the same length, either will do.
* You may use functions implemented above.
* You can iterate through the chain using `session.query(Block).all()`.
* Again, feel free to use ORM.

In [ ]:
# Test  
session = dbmanager.get_session()
block_set = session.query(Block).all()
longest_chain = dbmanager.get_longest()
if block_set == longest_chain:
    print("PASS")
else:
    print("FAIL")

### Problem 5 : `DBManager.search_transaction`

Implement `def search_transaction(self, tx_hash: str)` function in [dbmanager.py](/edit/blends/node/blockchain/dbmanager.py)

* Search the transaction by the hash value.
* Iterate throug the transactions using `session.query(Transaction).all()`.
* Again, feel free to use ORM.

In [ ]:
# Test  
session = dbmanager.get_session()
tx_set = session.query(Transaction).all()
for tx in tx_set:
    header = tx.get_header()
    if(tx == dbmanager.search_transaction(header["hash"])):
        print("PASS")
    else:
        print("FAIL")

### Problem 6 : `DBManager.get_block_balance`

* Now let's compute the balance of a block.
* The API must return `Dictionary` of `{ "key" : value }`.
* Iterate through the blocks using `session.query(Block).all()`.
* You should add `REWARD` to miner's account.
* Again, feel free to use ORM.

Implement `def get_block_balance(self, block_hash: str)` function in [verifier.py](/edit/blends/node/blockchain/dbmanager.py)

In [ ]:
# Test  
session = dbmanager.get_session()
block_set = session.query(Block).all()
miner_set = []
for block in block_set:
    if block.miner not in miner_set:
        miner_set.append(block.miner)
value_set = [54, 20, 26]

block = block_set[4]
header = block.get_header()
balance_dict = dbmanager.get_block_balance(header["hash"])
for i in range(3):
    if balance_dict[miner_set[i]] == value_set[i]:
        print("PASS")
    else:
        print("FAIL")

## Section 3: Verifier/Validator

Here you will implement function to verify correctness of transactions and blocks.

BLENDS employs two steps of `Verifier` and `Validator`.

* Verifier: verifies intra-transaction and intra-block their own correctness.
* Validator: veritifes intra-transaction and inter-block correctness.


In [ ]:
# Initialization
from blends.node.blockchain.verifier import Verifier
verifier = Verifier()

### Problem 1 : `Verifier.verify_transaction`

Implement `def verify_transaction(self, transaction: Transaction)` function in [verifier.py](/edit/blends/node/blockchain/verifier.py)

* This funciton checks in a transaction the following points: (1) all fields in transaction are in correct format; (2) the hash and the digital signature are correct.
* If the transaction passes all the checks, the function returns `True`; otherwise `False`.

In [ ]:
# Test  
import copy
session = dbmanager.get_session()
tx_set = []
tx_set = session.query(Transaction).all()
for tx in tx_set:
    if(verifier.verify_transaction(tx)):
        print("PASS")
    else:
        print("FAIL")

ill_tx_set = copy.deepcopy(tx_set)
ill_tx_set[0].hash = "0xdeadbeef"
ill_tx_set[1].receiver = ill_tx_set[1].sender
ill_tx_set[2].amount = 1000
ill_tx_set[3].timestamp = "2012-12-08T05:14:24"
for ill_tx in ill_tx_set:
    if(verifier.verify_transaction(ill_tx)):
        print("FAIL")
    else:
        print("PASS")

### Problem 2: `Verifier.verify_block`

Implement `def verify_block(self, block: Block)` function in [verifier.py](/edit/blends/node/blockchain/verifier.py)

* This function checks in a block the following points: (1) all fields are correct; (2) the hash is correct; (3) if the nonce is within the given range of `Difficulty`. (4) all transactions in the block are correct.
* Block hash must be smaller than target difficulty hash: 2^512/2^(20+difficulty)
* If the block passes all the checks, the function returns `True`; otherwise `False`.

In [ ]:
# Test  
session = dbmanager.get_session()
block_set = session.query(Block).all()
for block in block_set:
    if(verifier.verify_block(block)):
        print("PASS")
    else:
        print("FAIL")
        
ill_block_set = []
ill_block_set.append(copy.deepcopy(block_set[0]))
ill_block_set.append(copy.deepcopy(block_set[1]))
ill_block_set[0].hash = "0xdeadbeef"
ill_block_set[1].difficulty = 12
for ill_block in ill_block_set:
    if(verifier.verify_block(ill_block)):
        print("FAIL")
    else:
        print("PASS")

### Problem 3: `Blockchain.validate_transaction`

Implement `def validate_transaction(self, tx: Transaction, block: Block) ` function in [blockchain.py](/edit/blends/node/blockchain/blockchain.py)

* This function checks if the transaction can be added to a block.
* First, it must check if the transaction has been included in another block.
* Then, if the transction is added, the balance is correct.
* If the transaction is correct, the function returns `True`; otherwise, `False`.

In [ ]:
from blends.node.blockchain.blockchain import Blockchain
blockchain = Blockchain("sqlite:///scenario.db")
# Test  
session = dbmanager.get_session()
block_set = session.query(Block).all()
tx_set = session.query(Transaction).all()

# good transaction
new_tx = Transaction.new_transaction("beta", hex(sender.n), hex(receiver.n), "2018-12-08T08:28:38", 10)
payload = new_tx.get_payload()
hash_ = crypto.get_hash(payload)
new_tx.set_hash(hash_)
sign = "0x51a21ca0da4ee42169b37f948f8484b801c7f49270ab3e26dae3f487122c84bab7cd3d8a35b29ca550c2a46ddb11de7762b9b26f647efe710994950a74f79ca99a1aa10ae75f712b187f9becfbf1cc622b65465c5311fe69d1b699c31fb2b615c8625af4a2b90ed48aa1690633075491f44cbd2cf9b59d423d463432d881d78945befc537a3ba7e44918d0abf13c035b66deab8b3e10838bb5e2ccc69035e1d5b0b20f12ed0d8d7d96b403d9039d75faa960bcc2b2f53e37b6d457f54327f6e4122c79265535d9c8ff93aa769d4eae0e5ece25ec246a05b852695ef7bcad5d650ae66036db90c2d4e5b53ff30ec7d44ad40abc635559bdf4b7f78c51efe4245d"
new_tx.set_sign(sign)
# bad transaction
bad_tx = Transaction.new_transaction("beta", hex(sender.n), hex(receiver.n), "2018-12-08T08:28:38", 10000)
payload = bad_tx.get_payload()
hash_ = crypto.get_hash(payload)
bad_tx.set_hash(hash_)
sign = "0x92803837474863f65635d49f5f168541aab0e882a6d37864c8e5c3abf61e0ecb4bc465d6a01e34f309705e626b3ce3a666416e40eddb1ce6863eefb0db1fa41fb34c1ce60fe6b9cc4f96cd4b260daa6e8681fe7b86f8c5e985c9f6691444a5e2888de21d4c69a1d22b652569a3fbc13eaa2afc10feb9c92c9eaba6767b6e4d4e6e95ca61bb3fc009166ec95e12584f287034cc51d8abc20a07c468623ab238546512b830ce46968b254c113b22eabbea5b18f08a0ffe1c27f01bd0f2404267e568c87ccff2f58c2773e59efe39dba2729a917185f1b5efdd0f1dbb9d0d40db8ef9a40b5259a646ab30ed3579baa18ea7811d7b62098f3c931095b0a399dfd196"
bad_tx.set_sign(sign)

if blockchain.validate_transaction(new_tx, block_set[4]):
    print("PASS")
else:
    print("FAIL")
if blockchain.validate_transaction(bad_tx, block_set[4]):
    print("FAIL")
else:
    print("PASS")
if blockchain.validate_transaction(tx_set[2], block_set[4]):
    print("FAIL")
else:
    print("PASS")

### Problem 4: `Blockchain.validate_block`

Implement `def validate_block(self, block: Block)` function in [blockchain.py](/edit/blends/node/blockchain/blockchain.py)

* This function checks if the entire block is correct.
* It checks if the parent block exists.
* It checks if the balance of the block is all correct.
* If the block passes all the above checks, the function returns `True`; otherwise `False`.

In [ ]:
from blends.node.blockchain.blockchain import Blockchain
blockchain = Blockchain("sqlite:///scenario.db")
# Test  
session = dbmanager.get_session()
block_set = session.query(Block).all()
tx_set = session.query(Transaction).all()

for block in block_set:
    if blockchain.validate_block(block):
        print("PASS")
    else:
        print("FAIL")
        
block = Block.new_block("test", None, util.now(), hex(sender.n), 0)
if blockchain.validate_block(block) and not blockchain.verifier.verify_block(block):
    print("FAIL")
else:
    print("PASS")
     
block = Block.new_block("test", "0x12313", util.now(), hex(sender.n), 0)
if blockchain.validate_block(block) and not blockchain.verifier.verify_block(block):
    print("FAIL")
else:
    print("PASS")
    

ill_block_set = []
ill_block_set.append(copy.deepcopy(block_set[0]))
ill_block_set.append(copy.deepcopy(block_set[1]))
ill_block_set[0].parent = "0xdeadbeef"
ill_block_set[1].txs.append(Transaction.new_transaction("beta", hex(sender.n), hex(receiver.n), "2018-12-08T08:28:38", 10000)) 

for block in ill_block_set:
    if blockchain.validate_block(block) and not blockchain.verifier.verify_block(block):
        print("FAIL")
    else:
        print("PASS")



## Section 4: Mining

In this section, you will implement mining. 

BLENDS uses basic PoW (Proof-of-Work) based consensous algorithm. 

### Problem 1: `mine_block`

Implement`mine_block` function in [miner.py](/edit/blends/node/miner/miner.py) 

In the function, you have to generate a hash value which has higher difficulty than the `self.block`'s difficulty.


* You should update `timestamp` periodically.
* You can access `hash`, `nonce`, `timestamp`, `difficulty` by accessing `self.block.<field_name>`.
* `difficulty` is read-only value. You should update the others.
*  If a new block is mint, the function returns `True`
* In this test code, you mine a block with low difficulty.


In [ ]:
import threading

from blends.node.miner import Miner
from blends.node.model import Block, Transaction
from blends.node.blockchain.verifier import Verifier
from blends.node import crypto, util

key = crypto.load_secret_key('key0.json')
key1 = crypto.load_secret_key('key1.json')
key2 = crypto.load_secret_key('key2.json')

DIFFICULTY = 0

block = Block.new_block("testing", '0xdecaf', util.now(), hex(key.n), DIFFICULTY)

tx1 =  Transaction.new_transaction("testing", hex(key.n), hex(key1.n), util.now(), 1)
tx1.set_hash(crypto.get_hash(tx1.get_payload()))
tx1.set_sign(crypto.sign(key, tx1.hash))

tx2 =  Transaction.new_transaction("testing", hex(key.n), hex(key1.n), util.now(), 3)
tx2.set_hash(crypto.get_hash(tx2.get_payload()))
tx2.set_sign(crypto.sign(key, tx2.hash))

block.txs.append(tx1)
block.txs.append(tx2)

mining = threading.Event()
mining.set()

miner = Miner(block, mining)
print("Start Mining: be patient.")
if miner.mine_block():
    
    verifier = Verifier()
    if verifier.verify_block(miner.block):
        print("PASS", miner.block)
    else:
        print("FAIL")
        print(miner.block)
        print(miner.block.get_header())
        print(miner.block.get_payload())

print("Done")

### Problem 2: Mining a Real Block

Finally, you have to mine a new block with same dificulty in our blockchain.

* A new block will be inserted in the `scenario.db`. Database insertion requires addtional constraint checkings. If it fails, you should check formats of your model implementations.
* Mining a block with high difficulty consumes much time (hours). You should optimize your codes for faster mining.


In [ ]:
DIFFICULTY = 6


parent = blockchain.get_current()

block = Block.new_block("beta", parent.hash, util.now(), hex(key.n), DIFFICULTY)

tx1 =  Transaction.new_transaction("beta", hex(key.n), hex(key1.n), util.now(), 1)
tx1.set_hash(crypto.get_hash(tx1.get_payload()))
tx1.set_sign(crypto.sign(key, tx1.hash))

block.txs.append(tx1)

blockchain = Blockchain("sqlite:///scenario.db")


miner = Miner(block, mining)
print("Start Mining: be patient.")
if miner.mine_block():

    if blockchain.append(miner.block):
        print("PASS", miner.block)
    else:
        print("FAIL")


else:
    print("FAIL")
    print(miner.block)
    print(miner.block.get_header())
    print(miner.block.get_payload())

print("Done")
